# PPD: OpenMP

Hélio - DC/UFSCar - 2023

# Programação paralela com extensões de linguagem: OpenMP

Há algum tempo, o aumento de desempenho dos sistemas computacionais passou a ser buscado a partir da replicação de suas unidades funcionais, já que estava cada vez mais difícil simplesmente aumentar a velocidade dos processadores. 

Processadores com vários núcleos (*cores*) e múltiplos processadores passaram a ser utilizados em computadores servidores e até em computadores pessoais e smartphones!

Como resultado, programadores que buscam aumentar o desempenho de suas aplicações passaram a ter que paralelizar os códigos utilizados. Visto pela complexidade da [programação com a biblioteca de *pthreads*](https://computing.llnl.gov/tutorials/pthreads/), contudo, essa não é uma tarefa fácil. 

Por outro lado, também não é simples para um compilador detectar automaticamente quais atividades de um programa podem ser executadas em paralelo e ajustar o código para isso. Questões relacionadas a dependências de dados tanto podem fazer com que um programa tenha trechos paralelizados de maneira indevida, quanto que sejam mantidos sequenciais em situações em que o compilador não tem certeza da ausência de erros com a  paralelização.

Assim, uma estratégia intermediária foi considerada para a **transformação de código sequencial** existente **em código paralelo**: **usar dicas do programador**, na forma de marcas (**pragmas**) adicionadas ao código, e paralelização dos cógigos indicados, usando *threads*, feita pelo compilador.

**OpenMP** (*Open Multi-Processing* - http://openmp.org) é uma interface de programação (API) que possibilita o desenvolvimento de programas em C/C++ e Fortran para ambientes multiprocessados.

Definido por um grupo formado por grandes fabricantes de hardware e software, OpenMP é um modelo portável e escalável que provê uma interface simples e flexível para o desenvolvimento de aplicações paralelas para execução em computadores com memória compartilhada.

Diferentes arquiteturas são suportadas, variando de estações de trabalho a supercomputadores, incluindo plataformas Unix e Windows.

De maneira geral, OpenMP consiste em um conjunto de diretivas de compilação, em uma biblioteca de funções e em variáveis de ambiente que influenciam o comportamento da execução de programas.

# Modelo de programação

Originalmente, o modelo de programação oferecido por OpenMP é o de paralelismo baseado em *threads* para ambiente com memória compartilhada (*Shared Memory, Thread Based Paralelism*). Assim, o cenário típico para uso desse mecanismo são os computadores ***multicores***. 

    Versões mais recentes das especificações OPenMP também têm suporte para paralelismo 
    usando aceleradores e GPGPUs mas, por ora, trataremos de CPUs.

OpenMP permite:

* Criar times de *threads* para execução paralela de blocos de código
* Especificar como dividir (*share*) as atividades de um bloco de código entre os membros de um grupo
* Declarar variáveis compartilhadas e privadas
* Sincronizar *threads* e permitir que executem operações de maneira exclusiva
* Executar *loops* usando operações SIMD
* Utilizar dispositivos como GPUs para processamento vetorial

OpenMP suporta ambos os modelos de [decomposição](https://hpc.llnl.gov/training/tutorials/introduction-parallel-computing-tutorial#Designing) das atividades: decomposição de código (funcional) e decomposição dados (de domínio).

* Paralelismo das atividade com OpenMP é definido de maneira explícita, não automática, com controle total do programador.
* Paralelismo é especificado por diretivas de compilação (pragmas em C/C++ (*pragmatic information*)), que permitem passar informações ao compilador.
* Informações passadas pelos *pragmas* podem ser ignoradas pelo compilador sem alterar a correção do código gerado.
* Esforço de paralelização de um programa com OpenMP resume-se, em geral, à identificação do paralelismo e não à reprogramação do código para implementar o paralelismo desejado.

Diferentes compiladores têm suporte à programacão com OpenMP: [compilers and tools](https://www.openmp.org/resources/openmp-compilers-tools/).

# Compilando programas OpenMP com gcc

Como OpenMP trata de extensões de linguagem, o suporte ao seu uso também deve ser oferecido diretamente **pelo compilador** utilizado. Assim, não basta apenas incluir definições e ligar bibliotecas ao código, mas é preciso saber quais parâmetros são necessários pelo compilador em uso.

Diferentes implementações de compiladores C/C++ e Fortran têm suporte a OpenMP, o que inclui [gcc](https://gcc.gnu.org/wiki/openmp).

Para compilar e gerar código executável C com OpenMP em gcc:
```
$ gcc prog.c -o prog -fopenmp  ...         // compila programa, incluindo suporte para openmp
````
Para saber mais, vale ler o manual:
```
$ man gcc                // sempre bom consultar o manual :-)

  /openmp                // busca por openmp ...

-fopenmp 
  Enable handling of OpenMP directives "#pragma omp" in C/C++ ... 
  When -fopenmp is specified, the compiler generates parallel code according
  to the  OpenMP Application Program Interface v2.5 <http://www.openmp.org/>.
  This option implies -pthread, and thus is only supported on targets that 
  have support for -pthread.
```


Vale saber também que as especificações OpenMP foram evoluindo ao longo do tempo e incluindo novas funcionaliddes. Assim, pode ser relevante saber qual é a versão OpenMP suportada pelo compilador. 


https://www.openmp.org/specifications/
 
    OpenMP 5.2 Specification – Nov 2021
    OpenMP 5.1 Specification – Nov 2020
    OpenMP 5.0 Specification – Nov 2018
    OpenMP 4.5 Specification – Nov 2015
    OpenMP 4.0 Specification – Jul 2013
    OpenMP 3.1 Specification – Jul 2011


Usando o compilador *gcc*, por exemplo, pode-se saber a verão OpenMP suportada consultando-se um valor pré-definido pela biblioteca *incluída* no programa:

    The _OPENMP macro is defined by OpenMP-compliante implementations as the decimal constant
    yyyymm, which will be the year and month of the approved specification.

In [1]:
%%writefile version.c

#include <omp.h> 
#include <stdio.h> 

int 
main()
{
  /*
    https://www.openmp.org/specifications/

    The _OPENMP macro is defined by OpenMP-compliante implementations as the decimal constant
    yyyymm, which will be the year and month of the approved specification.
 
    OpenMP 5.2 Specification – Nov 2021
    OpenMP 5.1 Specification – Nov 2020
    OpenMP 5.0 Specification – Nov 2018
    OpenMP 4.5 Specification – Nov 2015
    OpenMP 4.0 Specification – Jul 2013
    OpenMP 3.1 Specification – Jul 2011
  */
 
#ifdef _OPENMP
  printf("Compiled by an OpenMP-compliant implementation: %d\n",_OPENMP);
#endif

  return 0;
}

Writing version.c


In [8]:
! if [ ! version -nt version.c ]; then gcc version.c -o version -fopenmp && ./version; fi
# ! clang version.c -o clang-ver -fopenmp -stdlib=libc++ -Xopenmp && ./clang-ver

# Paralelismo com OpenMP

De maneira geral, a paralelização de códigos com OpenMP é feita da seuinte forma: 

* programador insere diretivas (**#pragmas**) no código, indicando ao compilador qual linha ou bloco de código devem ser paralelizados e de qual forma;
* compilador identifica as diretivas e transforma o código sequencial em um código paralelo.

Diretivas (pragmas) são inseridas como linhas de código e aplicam-se à linha de código logo abaixo. Quando o objetivo é aplicar uma diretiva a um bloco de código, com várias linhas, é preciso envolver este bloco de código com o uso de chaves "{ ... }". 

As diretivas OpenMP especificadas como *pragmas* têm a seguinte sintaxe:

```
#pragma omp nome_da_diretiva [ cláusulas_da_diretiva ]
```

Como se observa, primitivas podem, opciohnalmente, ter **cláusulas** variadas, que definem aspectos do funcionamento da diretiva. 

Caso o compilador utilizado não tenha suporte a OpenMP, ou se o parâmetro ***-fopenmp*** não for especificado na invocação do compilador ***gcc***, por exemplo, as linhas contendo as diretivas (#pragmas) serão simplesmente ignoradas pelo compilador. 

<br>

Os exemplos a seguir mostram a paralelização de um bloco de código usando a diretiva ***parallel***, especificada como uma *pragma* no código. 


No exemplo abaixo, vemos o uso da diretiva *parallel* para criar um time de *threads*. As diretivas sempre se aplicam à linha de código imediatamente abaixo, ou ao bloco de código definido entre chaves \{ ... \} . Neste caso, tudo que está dentro do bloco de código definido a partir da linha seguinte à *pragma*, será executado por todas as *threads* do time. 

Nesse caso, vê-se também que a **diretiva *parallel*** tem uma cláusula, **num_threads(4)**, que especifica quantas *threads* deverão ser usadas para a execução paralela do bloco a seguir. 

Apenas uma *thread*, aquela que encontrou a construção paralela e criou o time de *threads*, prossegue em execução após a região paralela. Neste caso, era a *thread* associada à função *main*. Apenas essa *thread* prossegue em execução, de forma que o comando printf final neste caso só será executado uma vez.



In [ ]:
%%writefile p1.c

#include <stdio.h>

int main ()
{
  // ...   // Código serial, executado por apenas 1 thread, como usual
  // ...

  // Uso da diretiva parallel para criar uma região paralela:
  #pragma omp parallel num_threads(4)  
  {
    // Seção paralela, executada por todas as threads do time
    printf("Hello, world!\n"); 
  }
  // Ao fim do bloco de código da região paralela, a thread master espera pela conclusão das demais
  // Apenas thread master (aquela que encontrou a região paralela e criou o time) prossegue execução
  
  printf("Goodbye\n");
  // ...
  return 0;
}


Overwriting p1.c


In [ ]:
! gcc p1.c -o p1 -fopenmp && ./p1
! echo
# Observe que se o código for compilado sem o parâmetro "-fopenmp" as linhas com pragmas serão ignoradas e não haverá paralelismo
! gcc p1.c -o p1 && ./p1

Hello, world!
Hello, world!
Hello, world!
Hello, world!
Goodbye

Hello, world!
Goodbye


Neste segundo exemplo, repete-se a criação de um time de *threads* para execução paralela de um bloco de código. É importante observar que, como não foram especificadas quantas *threads* usar, o compilador tomará como base um valor *default*, que é igual ao número de núcleos de processamento (cores) existentes no sistema. O controle sobre o número de *threads* será tratado com mais detalhes posteriormente.

Outro aspecto a observar é o uso de 2 chamadas da API omp. Para usá-las, é preciso incluir o arquivo de cabeçalhos ***omp.h*** no código. 

A função ***omp_get_thread_num***() retorna o número lógico de cada *thread* dentro do time que está executando a região paralela. 

Já a função ***omp_get_num_threads***() indica quantas *threads* há no time atual. O conhecimento desses índices pode ser útil quando o programador deseja controlar explicitamente o que cada *thread* do time irá fazer.

In [ ]:
%%writefile p2.c

#include <stdio.h>
#include <omp.h>   // necessário apenas se formos usar funções da API omp

// ...

int main ()
{
  // ...
  // Início de seção paralela: geração das threads do time
  #pragma omp parallel 
  {
    // ...
    // Seção paralela, executada por todas as threads do time
    printf("Esta é a thread %d de um time de %d.\n", omp_get_thread_num(), omp_get_num_threads());
    // ...
  }  
  // Apenas master thread prossegue execução após o bloco paralelo
  // ...
  return 0;
}


Writing p2.c


In [ ]:
! gcc -Wall p2.c -o p2 -fopenmp && ./p2

Esta é a thread 0 de um time de 2.
Esta é a thread 1 de um time de 2.


Um aspecto importante na programação com *threads* é tratamento das variávies do programa. Como *threads* compartilham as áreas de memória do processo ao qual estão associadas, é preciso atenção para qual será o efeito da criação automática de várias *threads* no programa com relaçào às variáveis utilizadas.

No exemplo a seguir, ilustra-se a definição de escopo de variáveis dentro de um bloco paralelo. Por padrão, qualquer **variável global**, ou qualquer variável definida dentro da função *main* neste caso, que é acessível dentro do código da região paralela, será **compartilhada** por todas as *threads* do time. Isso significa que qualquer modificação de uma dessas variáveis ocorrerá sobre a única instância, compartilhada por todas as *threads* do processo.

Na criação de um time de *threads* com a diretiva ***parallel***, contudo, há cláusulas específicas que podem ser usadas para **definir o escopo** de variáveis: 

* a cláusula ***private***() indica uma lista de variáveis que serão **privadas**, ou seja, cada *thread* terá uma cópia dessas variáveis;
* já a cláusula ***shared***() indica que as variáveis listadas serão **compartilhadas** pelas *threads* do time. 

Por padrão, variáveis não mencionadas nas cláusulas são tratadas como **compartilhadas**. Assim, todas as referências a uma variável vão se referir à mesma posição de memória. Caso esse não seja o comportamento desejado, é preciso especificar esta variável na lista de variáveis privadas.

Variáveis que forem definidas dentro do bloco paralelo (o que é permitido se não for usada a sintaxe C ansi ou -std=c90) também serão privadas para cada thread. 


In [ ]:
%%writefile p3.c

#include <stdio.h>
#include <stdlib.h>  // para funcao rand()
#include <time.h>    // para funcao time()
#include <omp.h>     // necessário apenas se formos usar funções da API omp

int main ()
{
  int var1, var2;
  // ...
  srand(time(NULL));
  
  var2 = 0;
  
  // Região paralela com definição do escopo de variáveis usadas no trecho paralelo.
  #pragma omp parallel private(var1) shared(var2)
  {
    int ind;      // variáveis definidas dentro da região paralela são privadas para cada thread do time
    
    ind = omp_get_thread_num();   // obtem numero logico da thread e salva em índice local

    // ...
    var1 = rand(); // como var1 é privada, cada thread terá uma instância de variável com esse nome
    // ...
   
    // var2 =      // será que todas as threads podem atualizar essa variavel ao mesmo tempo?
    
    printf("Thread %d: var1 = %d\n", ind, var1);

    // ...
  }
 
  // Apenas master thread prossegue execução após o bloco paralelo
  printf("\nFim\n");
  
  // ...
  return 0;
}

Overwriting p3.c


In [ ]:
! gcc p3.c -o p3 -fopenmp && ./p3

Thread 0: var1 = 613363060
Thread 1: var1 = 627224178

Fim


##Quantas *threads* são usadas numa região paralela?

Número de *threads* numa região paralela é determinado por diversos fatores:

* Avaliação da cláusula IF 
* Ajuste da cláusula ***num_threads*** na diretiva
* Uso da função ***omp_set_num_threads()*** antes da diretiva parallel
* Ajuste da variável de ambiente **OMP_NUM_THREADS** antes de iniciar a execução do programa




In [ ]:
%%writefile p4.c

#include <stdio.h>
#include <stdlib.h>   // para rand()
#include <time.h>     // para time()
#include <omp.h>      // para funções OpenMP

int
main()
{
  int impar;
 
  #pragma omp parallel
  printf("Thread %d de %d\n",omp_get_thread_num(), omp_get_num_threads());
 
  printf("\n");
 
  #pragma omp parallel num_threads(4)
  printf("Thread %d de %d\n",omp_get_thread_num(), omp_get_num_threads());

  printf("\n");
 
  omp_set_num_threads(6);
 
  #pragma omp parallel
  printf("Thread %d de %d\n",omp_get_thread_num(), omp_get_num_threads());

  printf("\n");
 
  srand(time(NULL));
  impar = rand()%2;
 
  #pragma omp parallel if(impar)
  printf("Thread %d de %d\n",omp_get_thread_num(), omp_get_num_threads());
     
  return 0;
}

Overwriting p4.c


In [ ]:
! gcc -Wall p4.c -o p4 -fopenmp

In [ ]:
! lscpu | grep "CPU(s)" && \
 echo && echo "Executando sem ajustar OMP_NUM_THREADS" && echo "" && \
 ./p4 && \
 echo "" && echo "Executando com OMP_NUM_THREADS=3" && echo "" && \
 OMP_NUM_THREADS=3 ./p4

CPU(s):              2
On-line CPU(s) list: 0,1
NUMA node0 CPU(s):   0,1

Executando sem ajustar OMP_NUM_THREADS

Thread 0 de 2
Thread 1 de 2

Thread 1 de 4
Thread 3 de 4
Thread 2 de 4
Thread 0 de 4

Thread 5 de 6
Thread 2 de 6
Thread 0 de 6
Thread 3 de 6
Thread 1 de 6
Thread 4 de 6

Thread 1 de 6
Thread 3 de 6
Thread 0 de 6
Thread 2 de 6
Thread 5 de 6
Thread 4 de 6

Executando com OMP_NUM_THREADS=3

Thread 2 de 3
Thread 0 de 3
Thread 1 de 3

Thread 1 de 4
Thread 2 de 4
Thread 3 de 4
Thread 0 de 4

Thread 5 de 6
Thread 3 de 6
Thread 1 de 6
Thread 0 de 6
Thread 2 de 6
Thread 4 de 6

Thread 0 de 1
